In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import random
import os

In [3]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)

In [4]:
from poorman_transformer.data                 import TokenManager
from poorman_transformer.modeling.transformer import Transformer

In [5]:
input_file_path = "input.txt"
with open(input_file_path, 'r') as fh:
    data = fh.read()
token_lib = sorted(list(set(data)))
token_manager = TokenManager(token_lib)

In [6]:
print(''.join(token_lib))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
print(data[0:0+500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [8]:
# Define model
token_lib_size = len(token_lib)
embd_size      = 64
num_blocks     = 4
head_size      = 64 // 4
context_length = 32
num_heads      = embd_size // head_size

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Transformer(token_lib_size, embd_size, context_length, num_blocks, num_heads).to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')


0.210497 M parameters


In [9]:
# [[[ USER INPUT ]]]
timestamp_prev = "2023_0822_1233_05"
# epoch          = 240
epoch          = 240

drc_chkpt = "chkpts"
fl_chkpt_prev   = None if timestamp_prev is None else f"{timestamp_prev}.epoch_{epoch}.chkpt"
path_chkpt_prev = None if fl_chkpt_prev is None else os.path.join(drc_chkpt, fl_chkpt_prev)

chkpt = torch.load(path_chkpt_prev)

# Load weights...
model.module.load_state_dict(chkpt['model_state_dict']) if hasattr(model, 'module') else \
        model.load_state_dict(chkpt['model_state_dict'])

<All keys matched successfully>

In [10]:
def print_one_char(s, delay=0.05):
    import time
    for char in s:
        print(char, end='', flush=True)
        time.sleep(delay)
    print()

In [11]:
seed = 7
random.seed(seed)
torch.manual_seed(seed)

In [12]:
input = "X-ray Beam is "
input = input[-context_length:]
input_encoded = token_manager.encode(input[-context_length:])
input_encoded = torch.tensor(input_encoded)[None,].to(device)

print(f"___/ Input \___")
print(input)
print("")

prediction_length = 500
for _ in range(prediction_length):
    next_token_encoded = model.generate_one(input_encoded[:,-context_length:])
    input_encoded = torch.cat([input_encoded, next_token_encoded], dim = 1)

output = token_manager.decode(input_encoded.cpu().tolist()[0])
print(f"___/ Output \___")
# print(f"{output}")
print_one_char(output, delay = 0.01)

___/ Input \___
X-ray Beam is 

___/ Output \___
X-ray Beam is kindness.

BENVOLIO:
I will I myself look.

Clown:
Ho&s murder I
the woest been postern bends make you.

First Keeper:
In lay he is here it up and let me to my fathers,
Shall news of the wars of any his feet.
3 KING HENRY VI

DUKE OF AUMERLE:
Against pocts is, whose that thou art.

KING EDWARD IV:
Shall ever doth in the more of blood,
Forture, palace for 'tis now chief:
You have need thou proclame, talk he is a same too, sheeps, went a man,
Or cousin, well met, do
eye, no.

ISABELLA:
Good one me
